In [1]:
import pandas as pd
import numpy as np
import operator

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go

In [2]:
df = pd.read_csv('quake_db_1965-2016.csv')
df.head()

,Date,Time,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Type,...,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status,Place
0,01/02/1965,13:44:18,19.246,145.616,Earthquake,131.6,NaN,NaN,6.0,MW,...,NaN,NaN,NaN,NaN,ISCGEM860706,ISCGEM,ISCGEM,ISCGEM,Automatic,Unknown
1,01/04/1965,11:29:49,1.863,127.352,Earthquake,80.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,ISCGEM860737,ISCGEM,ISCGEM,ISCGEM,Automatic,"Ngidihopitonu, ID"
2,01/05/1965,18:05:58,-20.579,-173.972,Earthquake,20.0,NaN,NaN,6.2,MW,...,NaN,NaN,NaN,NaN,ISCGEM860762,ISCGEM,ISCGEM,ISCGEM,Automatic,"Pangai, TO"
3,01/08/1965,18:49:43,-59.076,-23.557,Earthquake,15.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,ISCGEM860856,ISCGEM,ISCGEM,ISCGEM,Automatic,Unknown
4,01/09/1965,13:32:50,11.938,126.427,Earthquake,15.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,ISCGEM860890,ISCGEM,ISCGEM,ISCGEM,Automatic,"Dapdap, PH"


In [3]:
country_code = 'IN'

In [4]:
country_df = df[df['Place'].str.contains(country_code)]
country_df.head()

,Date,Time,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Type,...,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status,Place
58,02/18/1965,04:26:37,25.011,94.186,Earthquake,55.0,NaN,NaN,5.6,MW,...,NaN,NaN,NaN,NaN,ISCGEM860307,ISCGEM,ISCGEM,ISCGEM,Automatic,"State of Manipur, IN"
402,04/04/1966,06:42:13,12.010,92.483,Earthquake,10.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,ISCGEM848084,ISCGEM,ISCGEM,ISCGEM,Automatic,Union Territory of Andaman and Nicobar Islands...
468,06/27/1966,10:59:19,29.706,80.935,Earthquake,25.0,NaN,NaN,6.3,MW,...,NaN,NaN,NaN,NaN,ISCGEM846871,ISCGEM,ISCGEM,ISCGEM,Automatic,"Dharchula, IN"
494,08/15/1966,02:15:32,28.565,78.961,Earthquake,27.0,NaN,NaN,5.6,MW,...,NaN,NaN,NaN,NaN,ISCGEM844491,ISCGEM,ISCGEM,ISCGEM,Automatic,"Shahabad, IN"
528,09/26/1966,05:10:57,27.419,92.576,Earthquake,14.4,NaN,NaN,5.9,MW,...,NaN,NaN,NaN,NaN,ISCGEM843878,ISCGEM,ISCGEM,ISCGEM,Automatic,"Bomdila, IN"


In [5]:
country_df = country_df[['Date', 'Latitude', 'Longitude', 'Magnitude', 'Depth', 'Type', 'Place']]
_, _, country_df['Year'] = country_df['Date'].str.split('/').str
country_df.loc[:, 'Year'] = country_df.loc[:, 'Year'].astype(int)

In [6]:
country_df.head()

,Date,Latitude,Longitude,Magnitude,Depth,Type,Place,Year
58,02/18/1965,25.011,94.186,5.6,55.0,Earthquake,"State of Manipur, IN",1965
402,04/04/1966,12.010,92.483,5.8,10.0,Earthquake,Union Territory of Andaman and Nicobar Islands...,1966
468,06/27/1966,29.706,80.935,6.3,25.0,Earthquake,"Dharchula, IN",1966
494,08/15/1966,28.565,78.961,5.6,27.0,Earthquake,"Shahabad, IN",1966
528,09/26/1966,27.419,92.576,5.9,14.4,Earthquake,"Bomdila, IN",1966


In [7]:
year_list = country_df['Year'].to_list()
year_dict = {}
for year in year_list:
    year_dict[year] = year_list.count(year)
print(year_dict)

{1965: 1, 1966: 4, 1967: 4, 1969: 1, 1970: 1, 1971: 2, 1973: 2, 1974: 2, 1977: 1, 1978: 2, 1979: 1, 1980: 2, 1981: 1, 1982: 1, 1984: 2, 1986: 2, 1987: 1, 1988: 1, 1989: 2, 1991: 2, 1993: 1, 1995: 1, 1997: 3, 1999: 2, 2000: 1, 2001: 4, 2002: 1, 2003: 4, 2004: 9, 2005: 5, 2006: 4, 2007: 1, 2008: 7, 2009: 1, 2010: 1, 2011: 1, 2012: 1, 2013: 2, 2014: 1, 2016: 1}


In [8]:
year_value = max(year_dict.items(), key=operator.itemgetter(1))[0]
year_value

2004

In [9]:
year_marks = np.linspace(1965, 2016, 15)
year_marks = [int(i) for i in year_marks]
year_marks

[1965,
 1968,
 1972,
 1975,
 1979,
 1983,
 1986,
 1990,
 1994,
 1997,
 2001,
 2005,
 2008,
 2012,
 2016]

In [10]:
external_scripts = ['https://www.google-analytics.com/analytics.js']
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__,
    external_scripts=external_scripts,
    external_stylesheets=external_stylesheets)
app.config['suppress_callback_exceptions'] = True

In [11]:
app.layout = html.Div([
    html.Div(id='map-update'),
    html.Div([
        dcc.Slider(
            id='year-slider',
            min=min(year_list),
            max=max(year_list),
            step=1,
            marks={i : '{}'.format(i) for i in year_marks},
            value=year_value
        )
    ], className='container')
])

In [12]:
def getDataYearValue(value):
    data = country_df[country_df['Year'] == value]
    return data

def PlotDensityMap(lat, lon, z, radius, colorscale):
    density_map_trace = go.Densitymapbox(
        lat=lat,
        lon=lon,
        z=z,
        radius=radius,
        colorscale=colorscale,
    )
    return density_map_trace

def LayoutDensity(height, width, mapbox_style, c_lat, c_lon, zoom):
    layout_density_map = go.Layout(
        height=height,
        width=width,
        autosize=True,
        showlegend=False,
        hovermode='closest',
        margin=dict(l=0, r=0, t=0, b=0),
        mapbox_style=mapbox_style,
        mapbox_center_lat=c_lat,
        mapbox_center_lon=c_lon,
        mapbox=dict(
            zoom=zoom
        )
    )
    return layout_density_map

In [13]:
@app.callback(
    Output('map-update', 'children'), [Input('year-slider', 'value')]
)
def output_update(value):
    country_df = getDataYearValue(value)
    center_lat = country_df[country_df['Magnitude'] == country_df['Magnitude'].max()]['Latitude'].to_list()[0]
    center_lon = country_df[country_df['Magnitude'] == country_df['Magnitude'].max()]['Longitude'].to_list()[0]
    mags = country_df['Magnitude'].to_list()
    lats = country_df['Latitude'].to_list()
    lons = country_df['Longitude'].to_list()
    map_trace = PlotDensityMap(lats, lons, mags, 10, 'Electric')
    layout_map = LayoutDensity(600, 980, 'stamen-terrain', center_lat, center_lon, 3)
    visualization = html.Div([
        dcc.Graph(
            id='density-map',
            figure={'data' : [map_trace], 'layout' : layout_map}
        ),
    ])
    return visualization    

In [14]:
# if __name__ == '__main__':
#     app.run_server()

In [15]:
def segregateYearQuakeCount(dataDf, countryCode):
    countryDf = dataDf[dataDf['Place'].str.contains(countryCode)]
    countryDf = countryDf[['Date', 'Latitude', 'Longitude', 'Magnitude', 'Depth', 'Type', 'Place']]
    _, _, countryDf['Year'] = countryDf['Date'].str.split('/').str
    countryDf.loc[:, 'Year'] = countryDf.loc[:, 'Year'].astype(int)
    yearRange = list(range(1965, 2017))
    yearDict = {}
    for year in yearRange:
        yearDf = countryDf[countryDf['Year'] == year]
        if yearDf.shape[0] == 0:
            yearDict[year] = 0
        else:
            yearDict[year] = yearDf.shape[0]
    return yearDict

In [16]:
yearDfCount = segregateYearQuakeCount(df, 'EC')
print(yearDfCount)

{1965: 1, 1966: 1, 1967: 3, 1968: 0, 1969: 1, 1970: 1, 1971: 3, 1972: 0, 1973: 3, 1974: 2, 1975: 0, 1976: 2, 1977: 0, 1978: 0, 1979: 2, 1980: 2, 1981: 2, 1982: 2, 1983: 4, 1984: 2, 1985: 0, 1986: 1, 1987: 7, 1988: 2, 1989: 1, 1990: 2, 1991: 2, 1992: 2, 1993: 0, 1994: 0, 1995: 6, 1996: 4, 1997: 0, 1998: 1, 1999: 4, 2000: 2, 2001: 0, 2002: 0, 2003: 1, 2004: 0, 2005: 10, 2006: 0, 2007: 3, 2008: 0, 2009: 0, 2010: 1, 2011: 2, 2012: 0, 2013: 0, 2014: 2, 2015: 1, 2016: 15}


In [17]:
def getNoneYear(dataDf, countryCode):
    countryDf = dataDf[dataDf['Place'].str.contains(countryCode)]
    countryDf = countryDf[['Date', 'Latitude', 'Longitude', 'Magnitude', 'Depth', 'Type', 'Place']]
    _, _, countryDf['Year'] = countryDf['Date'].str.split('/').str
    countryDf.loc[:, 'Year'] = countryDf.loc[:, 'Year'].astype(int)
    yearRange = list(range(1965, 2017))
    yearDict = {}
    for year in yearRange:
        yearDf = countryDf[countryDf['Year'] == year]
        if yearDf.shape[0] == 0:
            yearDict[year] = 0
    return yearDict

In [18]:
df = df[df['Place'] != 'Unknown']
df['CountryCode'] = df['Place'].apply(lambda x : x.split(', ')[len(x.split(', ')) - 1])

In [19]:
top_thirty = df['CountryCode'].value_counts().nlargest(30).index
top_thirty = list(top_thirty)
top_thirty.remove('PH')
top_thirty.remove('CL')
print(top_thirty)

['ID', 'PG', 'JP', 'VU', 'RU', 'US', 'SB', 'TO', 'MX', 'PE', 'CN', 'FJ', 'AR', 'TW', 'IR', 'AF', 'NZ', 'GR', 'GT', 'CO', 'TR', 'NI', 'KZ', 'EC', 'PK', 'IN', 'PA', 'MM']


In [20]:
for code in top_thirty:
    print('----------------- {} -------------------'.format(code))
    yearDfCountCode = getNoneYear(df, code)
    print(yearDfCountCode)
    print("\n")

----------------- ID -------------------
{}


----------------- PG -------------------
{}


----------------- JP -------------------
{}


----------------- VU -------------------
{}


----------------- RU -------------------
{}


----------------- US -------------------
{}


----------------- SB -------------------
{}


----------------- TO -------------------
{1969: 0}


----------------- MX -------------------
{}


----------------- PE -------------------
{}


----------------- CN -------------------
{}


----------------- FJ -------------------
{1969: 0, 2016: 0}


----------------- AR -------------------
{1986: 0, 1995: 0, 1996: 0}


----------------- TW -------------------
{2011: 0}


----------------- IR -------------------
{1967: 0, 1974: 0, 1984: 0, 1993: 0, 1995: 0, 2000: 0, 2009: 0, 2015: 0, 2016: 0}


----------------- AF -------------------
{1967: 0, 1970: 0, 1980: 0}


----------------- NZ -------------------
{1967: 0, 1969: 0, 1971: 0, 1972: 0, 1978: 0, 1979: 0, 1983: 0, 

In [23]:
df['Type'].value_counts()

Earthquake           14349
Nuclear Explosion      117
Explosion                1
Name: Type, dtype: int64

In [24]:
df.shape

(14467, 23)